In [113]:
import scvi
import scanpy as sc
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import anndata
import h5py    
import numpy as np   
import scipy.sparse as sparse
import scipy.io as sio
import scipy.stats as stats

In [114]:
## import dataset of 10x_AD raw data
feature = h5py.File("./data/Multiome_RNA_ATAC_Mouse_Brain_Alzheimers_AppNote_filtered_feature_bc_matrix.h5",'r+')    
feature_file = feature["matrix"]["features"]
ID = feature_file['id'][:]
modality = feature_file["feature_type"][:]
interval = feature_file["interval"][:]
name = feature_file["name"][:]
interval = [text.decode("utf-8")  for text in interval]
interval = [text.replace(':', '\t') for text in interval]
interval = [text.replace('-', '\t') for text in interval]
interval = [text.split('\t') for text in interval]
ID = [text.decode("utf-8")  for text in ID]
modality = [text.decode("utf-8")  for text in modality]
interval_pd = pd.DataFrame(interval)
table=np.array([ID, ID, modality, chro, start, end])
table2=table.transpose()
df = pd.DataFrame(table2)
df.to_csv("features.tsv", header=None, index=False, sep='\t')
data = feature["matrix"]
M1 = sparse.csc_matrix((data['data'][:],data['indices'][:], data['indptr'][:]), data['shape'][:])
sio.mmwrite("matrix.mtx",M1)
barcodes = [text.decode("utf-8")  for text in feature["matrix"]['barcodes'][:]]
barcodes = [text.replace('-', '_')  for text in barcodes]
barcodes = [text + '-'+text[text.find('_')+1:]  for text in barcodes]
barcodes = pd.DataFrame(barcodes)
barcodes.to_csv("data/10X_AD/barcodes.tsv", header=False, index=False)

NameError: name 'chro' is not defined

In [115]:
adata_multi = scvi.data.read_10x_multiome("data/10X_AD")

/home/haiyi/anaconda3/envs/halo/lib/python3.9/site-packages/scvi/data/_read.py:78: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return AnnData(data.tocsr(), var=features, obs=cell_annot)


In [116]:
adata_mvi = scvi.data.organize_multiome_anndatas(adata_multi)

In [23]:
scvi.model.MULTIVI.setup_anndata(adata_mvi, batch_key="modality")

In [24]:
vae = scvi.model.MULTIVI(adata_mvi, 32286, 66914)

In [26]:
vae.train(use_gpu=True, max_epochs=10)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/haiyi/anaconda3/envs/halo/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:385: LightningDeprecationWarning: The `Callback.on_epoch_end` hook was deprecated in v1.6 and will be removed in v1.8. Please use `Callback.on_<train/validation/test>_epoch_end` instead.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/10:   0%|                                                  | 0/10 [00:00<?, ?it/s]

/home/haiyi/anaconda3/envs/halo/lib/python3.9/site-packages/scvi/module/_multivae.py:557: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484775609/work/aten/src/ATen/native/TensorShape.cpp:2985.)
  x = torch.where(mask_expr.T, x_expr.T, x_acc.T).T


Epoch 10/10: 100%|████████████████| 10/10 [04:11<00:00, 25.13s/it, loss=7.68e+03, v_num=1]


In [105]:
import logging
from functools import partial
from typing import Dict, Iterable, List, Optional, Sequence, Union

import numpy as np
import pandas as pd
import torch
from anndata import AnnData
from scipy.sparse import csr_matrix, vstack
from torch.distributions import Normal

from scvi import REGISTRY_KEYS
from scvi._compat import Literal
from scvi._types import Number
from scvi._utils import _doc_params
from scvi.data import AnnDataManager
from scvi.data.fields import (
    CategoricalJointObsField,
    CategoricalObsField,
    LayerField,
    NumericalJointObsField,
    NumericalObsField,
)
from scvi.dataloaders import DataSplitter
from scvi.model._utils import (
    _get_batch_code_from_category,
    scatac_raw_counts_properties,
    scrna_raw_counts_properties,
)
from scvi.model.base import UnsupervisedTrainingMixin
from scvi.module import MULTIVAE
from scvi.train import AdversarialTrainingPlan, TrainRunner
from scvi.train._callbacks import SaveBestState
from scvi.utils._docstrings import doc_differential_expression, setup_anndata_dsp

from scvi.model import MULTIVI 

from scvi.model.base import BaseModelClass, VAEMixin
from scvi.model.base._utils import _de_core


In [111]:

class MultiVI_Parallel(MULTIVI):

    def __init__(
        self,
        adata: AnnData,
        n_genes: int,
        n_regions: int,
        n_hidden: Optional[int] = None,
        n_latent: Optional[int] = None,
        n_layers_encoder: int = 2,
        n_layers_decoder: int = 2,
        dropout_rate: float = 0.1,
        region_factors: bool = True,
        gene_likelihood: Literal["zinb", "nb", "poisson"] = "zinb",
        use_batch_norm: Literal["encoder", "decoder", "none", "both"] = "none",
        use_layer_norm: Literal["encoder", "decoder", "none", "both"] = "both",
        latent_distribution: Literal["normal", "ln"] = "normal",
        deeply_inject_covariates: bool = False,
        encode_covariates: bool = False,
        fully_paired: bool = False,
        **model_kwargs,
    ):
        super().__init__(adata,
        n_genes,
        n_regions,
        n_hidden,
        n_latent,
        n_layers_encoder,
        n_layers_decoder,
        dropout_rate,
        region_factors,
        gene_likelihood,
        use_batch_norm,
        use_layer_norm,
        latent_distribution,
        deeply_inject_covariates,
        encode_covariates,
        fully_paired,
        **model_kwargs,)

    @torch.no_grad()
    def get_latent_representation(
        self,
        adata: Optional[AnnData] = None,
        modality: Literal["joint", "expression", "accessibility"] = "joint",
        indices: Optional[Sequence[int]] = None,
        give_mean: bool = True,
        batch_size: Optional[int] = None,
    ) -> np.ndarray:
        """
        Return the latent representation for each cell.
        Parameters
        ----------
        adata
            AnnData object with equivalent structure to initial AnnData. If `None`, defaults to the
            AnnData object used to initialize the model.
        modality
            Return modality specific or joint latent representation.
        indices
            Indices of cells in adata to use. If `None`, all cells are used.
        give_mean
            Give mean of distribution or sample from it.
        batch_size
            Minibatch size for data loading into model. Defaults to `scvi.settings.batch_size`.
        Returns
        -------
        latent_representation : np.ndarray
            Low-dimensional representation for each cell
        """
        if not self.is_trained_:
            raise RuntimeError("Please train the model first.")

        keys = {"z": "z", "qz_m": "qz_m", "qz_v": "qz_v", "z_expr": "z_expr", 
        "qzm_expr": "qzm_expr", "qzv_expr": "qzv_expr", "z_acc": "z_acc", "qzm_acc": "qzm_acc", "qzv_acc": "qzv_acc"}
        

        adata = self._validate_anndata(adata)
        scdl = self._make_data_loader(
            adata=adata, indices=indices, batch_size=batch_size
        )
        latent = []
        latent_expr = []
        latent_atac = []
        for tensors in scdl:
            inference_inputs = self.module._get_inference_input(tensors)
            outputs = self.module.inference(**inference_inputs)
            qz_m = outputs[keys["qz_m"]]
            qz_v = outputs[keys["qz_v"]]
            z = outputs[keys["z"]]
            
            qzm_expr = outputs[keys["qzm_expr"]]
            qzv_expr = outputs[keys["qzv_expr"]]
            z_expr = outputs[keys["z_expr"]]

            qzm_acc = outputs[keys["qzm_acc"]]
            qzm_acc = outputs[keys["qzv_acc"]]
            z_acc = outputs[keys["z_acc"]]


            if give_mean:
                # does each model need to have this latent distribution param?
                if self.module.latent_distribution == "ln":
                    samples = Normal(qz_m, qz_v.sqrt()).sample([1])
                    z = torch.nn.functional.softmax(samples, dim=-1)
                    z = z.mean(dim=0)

                    samples_expr = Normal(qzm_expr, qzv_expr.sqrt()).sample([1])
                    z_expr = torch.nn.functional.softmax(samples_expr, dim=-1)
                    z_expr = z_expr.mean(dim=0)

                    samples_atac = Normal(qzm_acc, qzm_acc.sqrt()).sample([1])
                    z_acc = torch.nn.functional.softmax(samples_atac, dim=-1)
                    z_acc = z_acc.mean(dim=0)

                else:
                    z = qz_m
                    z_acc = qzm_acc
                    z_expr = qzm_expr

            
            latent += [z.cpu()]
            latent_atac+= [z_acc.cpu()]
            latent_expr += [z_expr.cpu()]

        return torch.cat(latent).numpy() , torch.cat(latent_atac).numpy(), torch.cat(latent_expr).numpy()


In [117]:
MultiVI_Parallel.setup_anndata(adata_mvi, batch_key="modality")

In [118]:
vae = MultiVI_Parallel(adata_mvi, 32286, 66914)

In [121]:
vae.train(epoch)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/haiyi/anaconda3/envs/halo/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:385: LightningDeprecationWarning: The `Callback.on_epoch_end` hook was deprecated in v1.6 and will be removed in v1.8. Please use `Callback.on_<train/validation/test>_epoch_end` instead.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/500:   0%|          | 0/500 [00:00<?, ?it/s]

/home/haiyi/anaconda3/envs/halo/lib/python3.9/site-packages/scvi/module/_multivae.py:557: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484775609/work/aten/src/ATen/native/TensorShape.cpp:2985.)
  x = torch.where(mask_expr.T, x_expr.T, x_acc.T).T
/home/haiyi/anaconda3/envs/halo/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
